# CS6910 Assignment 3 (RNN Frameworks for transliteration) - Without attention

In [1]:
# importing required libraries for the notebook

import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm
import wandb
import numpy as np
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import random
from language import *
from dataset_dataloader import *
from encoder_decoder import *

In [2]:
# set the device to 'cuda' if available
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Defining the source and target languages and loading data

In [3]:
# define the source and target languages
TARGET = 'hin'
SOURCE = 'eng'

In [4]:
# load all the available data and print sample counts for each set
x_train, y_train = load_data(TARGET, 'train')
x_valid, y_valid = load_data(TARGET, 'valid')
x_test, y_test = load_data(TARGET, 'test')

print(f'Number of train samples = {len(x_train)}')
print(f'Number of valid samples = {len(x_valid)}')
print(f'Number of test samples = {len(x_test)}')

Number of train samples = 51200
Number of valid samples = 4096
Number of test samples = 4096


In [5]:
# create language objects for storing vocabulary, index2sym and sym2index
SRC_LANG = Language(SOURCE)
TAR_LANG = Language(TARGET)

# creating vocabulary using train data only
SRC_LANG.create_vocabulary(*(x_train))
TAR_LANG.create_vocabulary(*(y_train))

# generate mappings from characters to numbers and vice versa
SRC_LANG.generate_mappings()
TAR_LANG.generate_mappings()

# print the source and target vocabularies
print(f'Source Vocabulary Size = {len(SRC_LANG.symbols)}')
print(f'Source Vocabulary = {SRC_LANG.symbols}')
print(f'Source Mapping {SRC_LANG.index2sym}')
print(f'Target Vocabulary Size = {len(TAR_LANG.symbols)}')
print(f'Target Vocabulary = {TAR_LANG.symbols}')
print(f'Target Mapping {TAR_LANG.index2sym}')

Source Vocabulary Size = 26
Source Vocabulary = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Source Mapping {0: '@', 1: '$', 2: '!', 3: '%', 4: 'a', 5: 'b', 6: 'c', 7: 'd', 8: 'e', 9: 'f', 10: 'g', 11: 'h', 12: 'i', 13: 'j', 14: 'k', 15: 'l', 16: 'm', 17: 'n', 18: 'o', 19: 'p', 20: 'q', 21: 'r', 22: 's', 23: 't', 24: 'u', 25: 'v', 26: 'w', 27: 'x', 28: 'y', 29: 'z'}
Target Vocabulary Size = 64
Target Vocabulary = ['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्']
Target Mapping {0: '@', 1: '$', 2: '!', 3: '%', 4: 'ँ', 5: 'ं', 6: 'ः', 7: 'अ', 8: 'आ', 9: 'इ', 10: 'ई', 11: 'उ', 12: 'ऊ', 13: 'ऋ', 14: 'ए', 15: 'ऐ', 16: 'ऑ', 17: 'ओ', 18: 'औ',

## Runner Class

In [6]:
class Runner():
    def __init__(self, src_lang : Language, tar_lang : Language, common_embed_size, common_num_layers, 
                 common_hidden_size, common_cell_type, enc_bidirect, dropout, opt_name='Adam',
                 learning_rate=1e-3):
        # save the language objects
        self.src_lang = src_lang
        self.tar_lang = tar_lang

        # create all the sub-networks and the main model
        self.encoder = EncoderNet(vocab_size=src_lang.get_size(), embed_size=common_embed_size,
                             num_layers=common_num_layers, hid_size=common_hidden_size,
                             cell_type= common_cell_type, bidirect=enc_bidirect, dropout=dropout)
        self.decoder = DecoderNet(vocab_size=tar_lang.get_size(), embed_size=common_embed_size,
                             num_layers=common_num_layers, hid_size=common_hidden_size,
                             cell_type=common_cell_type, dropout=dropout)
        self.model = EncoderDecoder(encoder=self.encoder, decoder=self.decoder, src_lang=src_lang, 
                                    tar_lang=tar_lang)
        
        # move model to the torch device
        self.model.to(device)
        # for reproducibility - seed everything with 42
        torch.manual_seed(42); torch.cuda.manual_seed(42); np.random.seed(42); random.seed(42)

        self.model.apply(self.init_weights) # initialize model weights

        # initialize the data loaders
        self.trainLoader, self.validLoader, self.testLoader = None, None, None

        # optimizer for the model and loss function [that ignores locs where target = PAD token]
        self.optimizer = None
        self.loss_criterion = nn.CrossEntropyLoss(ignore_index=tar_lang.sym2index[PAD_SYM])
        if opt_name == 'Adam':
            self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    @staticmethod
    def init_weights(m):
        '''
        function to initialize the weights of the model parameters
        '''
        for name, param in m.named_parameters():
            if 'weight' in name:
                nn.init.normal_(param.data, mean=0, std=0.05)
            else:
                nn.init.constant_(param.data, 0)
    
    def generate_data_loaders(self, data_X, data_y, batch_size):
        '''
        Create the pytorch Dataset and use it to make the dataloader. The dataloader
        post-processes every batch to add padding
        '''
        dataset = TransliterateDataset(data_X, data_y, src_lang=SRC_LANG, tar_lang=TAR_LANG)
        dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                                collate_fn=CollationFunction(SRC_LANG, TAR_LANG))
        return dataloader

    def make_all_loaders(self, train_data, valid_data, test_data, batch_size):
        ''' 
        get train, valid, test data and create dataloaders of batches with size=batch_size
        '''
        train_X, train_y = train_data
        valid_X, valid_y = valid_data
        test_X, test_y = test_data

        self.trainLoader = self.generate_data_loaders(train_X, train_y, batch_size)
        self.validLoader = self.generate_data_loaders(valid_X, valid_y, batch_size)
        self.testLoader = self.generate_data_loaders(test_X, test_y, batch_size)

    def get_accuracy(self, pred_words, tar_words):
        ''' 
        compute the accuracy using (predicted words, target words) and return it.
        exact word matching is used.
        '''
        assert(len(pred_words) == len(tar_words))
        count = 0
        for i in range(len(pred_words)):
            if pred_words[i] == tar_words[i]:
                count += 1
        return count / len(pred_words)

    def train(self, epoch_number, tf_ratio=0.6):
        '''
        train the model for 1 epoch with teacher forcing ratio = tf_ratio.
        epoch_number is displayed in the output
        '''
        # sanity checks 
        assert(self.trainLoader != None); assert(self.optimizer != None)

        # set model in training mode for autograd to be activated
        self.model.train(); self.optimizer.zero_grad()
        # maintain training loss and lists of predicted and true words
        train_loss = 0.0
        pred_words, true_words = [], []
        with tqdm(self.trainLoader, unit=' batch') as tqdmLoader:
            for batch_X, batch_y, X_lens, y_words in tqdmLoader:
                tqdmLoader.set_description(f'Epoch {epoch_number}')
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                # get the logits, preds for the current batch
                logits, preds = self.model(batch_X, batch_y, X_lens, tf_ratio=tf_ratio)
                # ignore loss for the first time step
                targets = batch_y[:, 1:]; logits = logits[:, 1:, :]
                logits = logits.swapaxes(1, 2) # make class logits the second dimension as needed
                loss = self.loss_criterion(logits, targets); loss.backward(); train_loss += loss.item()
                self.optimizer.step(); self.optimizer.zero_grad()
                batch_pred_words = self.tar_lang.convert_to_words(preds.cpu().numpy())
                tqdmLoader.set_postfix(loss=loss.item())
                true_words += y_words; pred_words += batch_pred_words
        train_loss /= len(self.trainLoader); train_acc = self.get_accuracy(pred_words, true_words)
        print(f'Train Loss = {train_loss}; Train Accuracy = {train_acc * 100}')
        return train_loss, train_acc, pred_words, true_words

    def evaluate(self):
        assert(self.validLoader != None)
        self.model.eval()
        valid_loss = 0.0
        true_words, pred_words = [], []
        with torch.no_grad():
            with tqdm(self.validLoader, unit= ' batch') as tqdmLoader:
                for batch_X, batch_y, X_lens, y_words in tqdmLoader:
                    tqdmLoader.set_description(f'Validation')
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                    # get the logits, preds for the current batch
                    logits, preds = self.model(batch_X, batch_y, X_lens)
                    # ignore loss for the first time step
                    targets = batch_y[:, 1:]; logits = logits[:, 1:, :]
                    logits = logits.swapaxes(1, 2) # make class logits the second dimension as needed
                    loss = self.loss_criterion(logits, targets)
                    valid_loss += loss.item()
                    batch_pred_words = self.tar_lang.convert_to_words(preds.cpu().numpy())
                    tqdmLoader.set_postfix(loss=loss.item())
                    true_words += y_words
                    pred_words += batch_pred_words
        valid_loss /= len(self.validLoader)
        valid_acc = self.get_accuracy(pred_words, true_words)
        print(f'Validation Loss = {valid_loss}; Validation Accuracy = {valid_acc * 100}')
        return valid_loss, valid_acc, pred_words, true_words

    def inference(self):
        assert(self.testLoader != None)
        self.model.eval()
        pred_words, true_words = [], []
        with torch.no_grad():
            with tqdm(self.testLoader, unit=' batch') as tqdmLoader:
                for batch_X, _, X_lens, y_words in tqdmLoader:
                    tqdmLoader.set_description(f'Testing')
                    batch_X = batch_X.to(device)
                    # get the predicted words for the current batch
                    batch_pred_words = self.model.inference_forward(batch_X, X_lens, 25)
                    true_words += y_words
                    pred_words += batch_pred_words
        test_acc = self.get_accuracy(pred_words, true_words)
        print(f'Test Accuracy = {test_acc * 100}')
        return pred_words, true_words
    
    def beam_search_inference(self, ):
        pass

In [10]:
# testing runner
# keep embedding small (around 32) -> important to get dense embedding
# also, adjust learning rate reasonably
runner = Runner(SRC_LANG, TAR_LANG, 32, 1, 128, 'LSTM', True, 0.1, learning_rate=1e-3)

In [11]:
runner.make_all_loaders((x_train, y_train), (x_valid, y_valid), (x_test, y_test), 128)

In [12]:
for i in range(10):
    _, _, p, w = runner.train(i, 0.6)
    runner.evaluate(); runner.inference()
    for x, y in list(zip(p,w))[:10]:
        print(x,y)

Epoch 0:  46%|████▋     | 186/400 [00:08<00:09, 21.87 batch/s, loss=3.05]


KeyboardInterrupt: 

In [167]:
for i in range(10):
    _, _, p, w = runner.train(i, 0.2)
    runner.evaluate(); runner.inference()
    for x, y in list(zip(p,w))[:10]:
        print(x,y)

Epoch 0: 100%|██████████| 400/400 [00:50<00:00,  7.90batch/s, loss=0.456]


Train Loss = 0.5901363408565521; Train Accuracy = 39.658203125


Validation: 100%|██████████| 32/32 [00:01<00:00, 24.37batch/s, loss=0.918]


Validation Loss = 1.1834038645029068; Validation Accuracy = 26.416015625


Testing: 100%|██████████| 32/32 [00:01<00:00, 20.94batch/s]


...

Train Loss = 0.22027455788105726; Train Accuracy = 69.396484375


Validation: 100%|██████████| 32/32 [00:01<00:00, 22.08batch/s, loss=1.39]


Validation Loss = 1.4701313748955727; Validation Accuracy = 27.7587890625


Testing: 100%|██████████| 32/32 [00:01<00:00, 20.04batch/s]

Test Accuracy = 23.681640625
देशभरामध्ये देशभरामध्ये
पेशनधारियों पेशनधारियों
अनुक्रमानुपतट अनुक्रमानुपात
वाहिन्यांसाठी वाहिन्यांसाठी
लोकरचनाओं लोकरचनाओं
ट्रांसीक्रि्टट ट्रांसक्रिप्टेज़
गंतिविधियों गंतिविधियों
शासनाबद्दल शासनाबद्दल
सर्वसंग्रह सर्वसंग्रह
तुमच्यापैकी तुमच्यापैकी


In [168]:
for i in range(10):
    _, _, p, w = runner.train(i, 0.)
    runner.evaluate(); runner.inference()
    for x, y in list(zip(p,w))[:10]:
        print(x,y)

Epoch 0: 100%|██████████| 400/400 [00:50<00:00,  7.95batch/s, loss=0.247]


Train Loss = 0.2603056246042252; Train Accuracy = 70.41796875


Validation: 100%|██████████| 32/32 [00:01<00:00, 24.44batch/s, loss=1.31]


Validation Loss = 1.4126202054321766; Validation Accuracy = 25.6591796875


Testing: 100%|██████████| 32/32 [00:01<00:00, 20.18batch/s]


...

Train Loss = 0.1637278700247407; Train Accuracy = 76.87890625


Validation: 100%|██████████| 32/32 [00:01<00:00, 24.71batch/s, loss=1.29]


Validation Loss = 1.5660292208194733; Validation Accuracy = 26.07421875


Testing: 100%|██████████| 32/32 [00:01<00:00, 20.88batch/s]

Test Accuracy = 23.53515625
देशभरामध्ये देशभरामध्ये
पेशनधारियों पेशनधारियों
अनुक्रमानुपात अनुक्रमानुपात
वाहिन्यांसाठी वाहिन्यांसाठी
लोकरचनाओं लोकरचनाओं
ट्रांसक्रप््टेज़ ट्रांसक्रिप्टेज़
गंतिविधियों गंतिविधियों
शासनाबद्दल शासनाबद्दल
सर्वसंग्रह सर्वसंग्रह
तुमच्यापैकी तुमच्यापैकी
